# 电影推荐系统

## 数据加载与预处理

使用了 pandas 和 numpy 库来处理数据和计算。

加载了两个数据集：
douban_users.csv 包含了用户的评分数据。
douban_movies.csv 包含了电影的详细信息。

数据预处理：

对用户数据 user_df 进行处理，仅保留了 user_id、movie_id 和 rating 三列。
对电影数据 movies_df 进行处理，选择了部分列并进行了重命名和去重操作。

映射关系构建：

使用了字典推导式来创建映射关系

用户-物品矩阵构建：

data_matrix：初始化一个二维数组，用来表示用户与电影之间的评分关系。

## 相似度计算

余弦相似度计算：
使用 cosine_similarity 函数计算了电影之间的余弦相似度，即 item_similarity 矩阵。

## 推荐功能实现

热门推荐 (rec_hot_movies)：

根据电影的评分和上映时间进行筛选，推荐近期评分较高的电影。

基于内容的推荐 (Recommend)：

给定一个电影ID，通过计算其与其他电影的相似度，返回最相似的前 k 部电影。

用户个性化推荐 (find_user_like)：

给定用户ID，首先找出用户喜欢的电影（评分大于等于4分），然后随机选择其中的一些电影，再基于这些电影进行推荐。

In [4]:


import random
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 加载用户数据
user_df = pd.read_csv('Data/douban_users.csv')
user_df = user_df.iloc[:, [1, 2, 3]]

# 加载电影数据
movies_df = pd.read_csv('Data/douban_movies.csv', encoding='utf-8')
movies_df = movies_df.iloc[:, [0, 1, 5, 6, 7, 8, 9, 12, 15, 16]]
movies_df = movies_df.drop_duplicates(subset='url')
movies_df = movies_df.rename(columns={'Unnamed: 0': 'Movie_ID'})

# 用户ID映射
usersMap = {user_id: idx for idx, user_id in enumerate(user_df['user_id'].unique())}

# 电影ID映射
moviesMap = {movie_id: idx for idx, movie_id in enumerate(movies_df['dataID'].unique())}

n_users = len(usersMap)
n_movies = len(moviesMap)

data_matrix = np.zeros((n_users, n_movies))  # 创建用户-电影评分矩阵

# 填充用户-电影评分矩阵
for idx, row in user_df.iterrows():
    try:
        data_matrix[usersMap[row['user_id']], moviesMap[row['movie_id']]] = row['rating']
    except KeyError:
        pass

# 使用余弦相似度计算电影之间的相似度
item_similarity = cosine_similarity(data_matrix.T)

def get_hot_movies():
    """
    获取热门推荐电影
    """
    hot_movies = movies_df[(movies_df['date'] >= 2019) & (movies_df['rate'] >= 8.7)]
    hot_movies = hot_movies[['name', 'actors', 'style1', 'rate', 'url']].values.tolist()
    return hot_movies

def recommend_similar_movies(movie_id, k=5):
    """
    根据给定的电影ID推荐相似的电影
    """
    recommended_movies = []
    try:
        movie_idx = moviesMap[movie_id]
        similar_indices = np.argsort(-item_similarity[movie_idx])[1:k + 1]

        for idx in similar_indices:
            rec_movie = movies_df.iloc[idx]
            rec_movie_info = [rec_movie['name'], rec_movie['actors'], rec_movie['style1'], rec_movie['rate'], rec_movie['url']]
            recommended_movies.append(rec_movie_info)
    except KeyError:
        pass

    return recommended_movies

def find_user_preferences(user_id):
    """
    查找用户喜欢的电影
    """
    user_seen_movies = user_df[user_df['user_id'] == user_id]['movie_id']
    user_like_movies = user_seen_movies[user_df['rating'] >= 4].sample(min(5, len(user_seen_movies)), replace=False)

    recommended_movies = []
    for movie_id in user_like_movies:
        recommendations = recommend_similar_movies(movie_id, k=7)
        recommended_movies.extend(recommendations)

    return recommended_movies

if __name__ == '__main__':
    hot_movies = get_hot_movies()
    print("热门推荐电影:", hot_movies)
    
    user_id = 'lmg'  # 替换为实际的用户ID
    user_preferences = find_user_preferences(user_id)
    print(f"用户 {user_id} 的偏好推荐:", user_preferences)


热门推荐电影: [['千与千寻', '柊瑠美  入野自由  夏木真理  菅原文太  中村彰男', '剧情', 9.4, 'https://movie.douban.com/subject/1291561/'], ['海上钢琴师', '蒂姆·罗斯  普路特·泰勒·文斯  比尔·努恩  克兰伦斯·威廉姆斯三世  梅兰尼·蒂埃里', '剧情', 9.3, 'https://movie.douban.com/subject/1292001/'], ['绿皮书', '维果·莫腾森  马赫沙拉·阿里  琳达·卡德里尼  塞巴斯蒂安·马尼斯科  迪米特·D·马里诺夫', '剧情', 8.9, 'https://movie.douban.com/subject/27060077/'], ['寄生虫', '宋康昊  李善均  赵汝贞  崔宇植  朴素丹', '剧情', 8.7, 'https://movie.douban.com/subject/27010768/'], ['何以为家', '赞恩·阿尔·拉菲亚  约丹诺斯·希费罗  博鲁瓦蒂夫·特雷杰·班科尔  卡萨尔·艾尔·哈达德  法迪·尤瑟夫', '剧情', 9.1, 'https://movie.douban.com/subject/30170448/'], ['小丑', '华金·菲尼克斯  罗伯特·德尼罗  马克·马龙  莎姬·贝兹  谢伊·惠格姆', '剧情', 8.7, 'https://movie.douban.com/subject/27119724/'], ['美丽人生', '罗伯托·贝尼尼  尼可莱塔·布拉斯基  乔治·坎塔里尼  朱斯蒂诺·杜拉诺  赛尔乔·比尼·布斯特里克', '剧情', 9.5, 'https://movie.douban.com/subject/1292063/'], ['海蒂和爷爷', '阿努克·斯特芬  布鲁诺·甘茨  昆林·艾格匹  安娜·申茨  伊莎贝尔·奥特曼', '剧情', 9.2, 'https://movie.douban.com/subject/25958717/'], ['波西米亚狂想曲', '拉米·马雷克  本·哈迪  约瑟夫·梅泽罗  格威利姆·李  艾伦·里奇', '剧情', 8.7, 'https://movie.douban.com/subject/53000

In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 加载用户数据
user_df = pd.read_csv('Data/douban_users.csv')
user_df = user_df.iloc[:, [1, 2, 3]]

# 加载电影数据
movies_df = pd.read_csv('Data/douban_movies.csv', encoding='utf-8')
movies_df = movies_df.iloc[:, [0, 1, 5, 6, 7, 8, 9, 12, 15, 16]]
movies_df = movies_df.drop_duplicates(subset='url')
movies_df = movies_df.rename(columns={'Unnamed: 0': 'Movie_ID'})

# 用户ID映射
usersMap = {user_id: idx for idx, user_id in enumerate(user_df['user_id'].unique())}

# 电影ID映射
moviesMap = {movie_id: idx for idx, movie_id in enumerate(movies_df['dataID'].unique())}

n_users = len(usersMap)
n_movies = len(moviesMap)

data_matrix = np.zeros((n_users, n_movies))  # 创建用户-电影评分矩阵

# 填充用户-电影评分矩阵
for idx, row in user_df.iterrows():
    try:
        data_matrix[usersMap[row['user_id']], moviesMap[row['movie_id']]] = row['rating']
    except KeyError:
        pass

# 使用余弦相似度计算电影之间的相似度
item_similarity = cosine_similarity(data_matrix.T)

def get_hot_movies(num_movies=10):
    """
    获取热门推荐电影
    """
    hot_movies = movies_df[(movies_df['date'] >= 2019) & (movies_df['rate'] >= 8.7)]
    hot_movies = hot_movies[['name', 'actors', 'style1', 'rate', 'url']].head(num_movies).values.tolist()
    return hot_movies

def recommend_similar_movies(movie_id, k=5):
    """
    根据给定的电影ID推荐相似的电影
    """
    recommended_movies = []
    try:
        movie_idx = moviesMap[movie_id]
        similar_indices = np.argsort(-item_similarity[movie_idx])[1:k + 1]
        #利用余弦相似度矩阵item_similarity，找到与给定电影最相似的电影。
#         item_similarity[movie_idx]返回该电影与所有其他电影的相似度向量。
# np.argsort(-item_similarity[movie_idx])按相似度降序排序索引，通过负号-实现降序。
#         [1:k + 1]选择前k个最相似的电影索引（排除第一个，因为第一个是电影本身）

        for idx in similar_indices:
            rec_movie = movies_df.iloc[idx]
            rec_movie_info = [rec_movie['name'], rec_movie['actors'], rec_movie['style1'], rec_movie['rate'], rec_movie['url']]
            recommended_movies.append(rec_movie_info)
    except KeyError:
        pass

    return recommended_movies

def find_user_preferences(user_id, num_movies=5):
    """
    查找用户喜欢的电影
    """
    user_seen_movies = user_df[user_df['user_id'] == user_id]['movie_id']
    user_like_movies = user_seen_movies[user_df['rating'] >= 4].sample(min(num_movies, len(user_seen_movies)), replace=False)

    recommended_movies = []
    for movie_id in user_like_movies:
        recommendations = recommend_similar_movies(movie_id, k=7)
        recommended_movies.extend(recommendations)

    return recommended_movies

# if __name__ == '__main__':
#     hot_movies = get_hot_movies(num_movies=10)
#     print("热门推荐电影:")
#     for movie in hot_movies:
#         print(movie)
    
#     user_id = 'lmg'  # 替换为实际的用户ID
#     user_preferences = find_user_preferences(user_id, num_movies=5)
#     print(f"用户 {user_id} 的偏好推荐:")
#     for movie in user_preferences:
#         print(movie)

if __name__ == '__main__':
    hot_movies = get_hot_movies(num_movies=5)
    print("热门推荐电影:")
    for idx, movie in enumerate(hot_movies, 1):
        print(f"{idx}.")
        print(f"电影名称: {movie[0]}")
        print(f"演员: {movie[1]}")
        print(f"类型: {movie[2]}")
        print(f"评分: {movie[3]}")
        print(f"链接: {movie[4]}")
        print()

    user_id = 'lmg'  # 替换为实际的用户ID
    user_preferences = find_user_preferences(user_id, num_movies=5)
    print(f"用户 {user_id} 的偏好推荐:")
    for idx, movie in enumerate(user_preferences, 1):
        print(f"{idx}.")
        print(f"电影名称: {movie[0]}")
        print(f"演员: {movie[1]}")
        print(f"类型: {movie[2]}")
        print(f"评分: {movie[3]}")
        print(f"链接: {movie[4]}")
        print()


热门推荐电影:
1.
电影名称: 千与千寻
演员: 柊瑠美  入野自由  夏木真理  菅原文太  中村彰男
类型: 剧情
评分: 9.4
链接: https://movie.douban.com/subject/1291561/

2.
电影名称: 海上钢琴师
演员: 蒂姆·罗斯  普路特·泰勒·文斯  比尔·努恩  克兰伦斯·威廉姆斯三世  梅兰尼·蒂埃里
类型: 剧情
评分: 9.3
链接: https://movie.douban.com/subject/1292001/

3.
电影名称: 绿皮书
演员: 维果·莫腾森  马赫沙拉·阿里  琳达·卡德里尼  塞巴斯蒂安·马尼斯科  迪米特·D·马里诺夫
类型: 剧情
评分: 8.9
链接: https://movie.douban.com/subject/27060077/

4.
电影名称: 寄生虫
演员: 宋康昊  李善均  赵汝贞  崔宇植  朴素丹
类型: 剧情
评分: 8.7
链接: https://movie.douban.com/subject/27010768/

5.
电影名称: 何以为家
演员: 赞恩·阿尔·拉菲亚  约丹诺斯·希费罗  博鲁瓦蒂夫·特雷杰·班科尔  卡萨尔·艾尔·哈达德  法迪·尤瑟夫
类型: 剧情
评分: 9.1
链接: https://movie.douban.com/subject/30170448/

用户 lmg 的偏好推荐:
1.
电影名称: 半生缘
演员: 黎明  吴倩莲  梅艳芳  黄磊  葛优
类型: 剧情
评分: 7.8
链接: https://movie.douban.com/subject/1299624/

2.
电影名称: 我的天才女友
演员: 埃琳娜·费兰特
类型: 剧情
评分: 9.5
链接: https://movie.douban.com/subject/30395843/

3.
电影名称: 神奇女侠
演员: 盖尔·加朵  克里斯·派恩  康妮·尼尔森  罗宾·怀特  丹尼·赫斯顿
类型: 动作
评分: 7.1
链接: https://movie.douban.com/subject/1578714/

4.
电影名称: 大象
演员: 阿里克斯·弗罗斯特  约翰·罗宾森  埃里克·德伦  伊里斯·麦康利  艾丽西亚·迈尔斯
类型: 剧